In [4]:
import numpy as np

# 경로 설정
cfg_path = "/home/avees/tee/tz_datasets/cfg/mnist_lenet.cfg"
weights_path = "mnist_lenet.weights"

# .cfg 파서
def parse_cfg(cfg_file):
    layers = []
    with open(cfg_file, "r") as f:
        lines = [l.strip() for l in f if l.strip() and not l.startswith("#")]
    layer = None
    for line in lines:
        if line.startswith("["):
            if layer: layers.append(layer)
            layer = {"type": line.strip("[]")}
        else:
            if "=" in line:
                k, v = [x.strip() for x in line.split("=", 1)]
                layer[k] = v
    if layer: layers.append(layer)
    return layers

# weights 로드
with open(weights_path, "rb") as f:
    header = np.fromfile(f, dtype=np.int32, count=5)
    weights = np.fromfile(f, dtype=np.float32)

print("📄 Header:", header)
print(f"총 파라미터 개수: {len(weights)}\n")

cfg_layers = parse_cfg(cfg_path)

idx = 0
prev_filters = 3  # cfg에 명시된 channels=3

for layer in cfg_layers:
    t = layer["type"]

    if t == "convolutional":
        filters = int(layer["filters"])
        size = int(layer["size"])
        bias_size = filters
        weight_size = filters * prev_filters * size * size

        bias = weights[idx : idx + bias_size]; idx += bias_size
        w = weights[idx : idx + weight_size]; idx += weight_size

        print(f"🧩 CONV ({filters} filters, {prev_filters} in, {size}x{size})")
        print(f"  bias shape: ({bias_size})")
        print(f"  weights shape: ({filters}, {prev_filters}, {size}, {size})")
        print(f"  bias (앞 10개): {bias[:10]}")
        print(f"  weights (앞 10개): {w[:10]}\n")

        prev_filters = filters

    elif t == "connected":
        out_neurons = int(layer["output"])
        bias_size = out_neurons
        weight_size = prev_filters * out_neurons

        bias = weights[idx : idx + bias_size]; idx += bias_size
        w = weights[idx : idx + weight_size]; idx += weight_size

        print(f"🧩 FC ({prev_filters} → {out_neurons})")
        print(f"  bias shape: ({bias_size})")
        print(f"  weights shape: ({out_neurons}, {prev_filters})")
        print(f"  bias (앞 10개): {bias[:10]}")
        print(f"  weights (앞 10개): {w[:10]}\n")

        prev_filters = out_neurons

# 최종 확인
print(f"총 사용된 파라미터: {idx} / {len(weights)} ✅")


📄 Header: [      0       2       0 3600128       0]
총 파라미터 개수: 47776

🧩 CONV (6 filters, 3 in, 5x5)
  bias shape: (6)
  weights shape: (6, 3, 5, 5)
  bias (앞 10개): [-0.33900645  0.0525151   0.00263852 -0.18520221  0.00416441 -0.00181954]
  weights (앞 10개): [-0.18868001 -0.06433833 -0.07672578 -0.26909724  0.10311449 -0.07641959
 -0.10905516  0.05266197  0.44004533  0.00901392]

🧩 CONV (6 filters, 6 in, 5x5)
  bias shape: (6)
  weights shape: (6, 6, 5, 5)
  bias (앞 10개): [-0.00967182 -0.31944704  0.04704136 -0.03584529 -0.30567467 -0.03402576]
  weights (앞 10개): [-0.0285754  -0.35232884 -0.10530736  0.22695127  0.4728608   0.08710439
 -0.1368281   0.0898189   0.16912338  0.2902037 ]

🧩 FC (6 → 120)
  bias shape: (120)
  weights shape: (120, 6)
  bias (앞 10개): [-0.01848637 -0.03754127 -0.02873765 -0.03125414 -0.09974167  0.02318632
  0.02022447  0.04628567  0.01646404 -0.06894489]
  weights (앞 10개): [ 0.06081098 -0.0805511  -0.03713055 -0.0421381   0.05067382  0.00871376
 -0.06854074 -0.

In [5]:
import numpy as np

weights_path = "mnist_lenet.weights"

with open(weights_path, "rb") as f:
    header = np.fromfile(f, dtype=np.int32, count=5)
    weights = np.fromfile(f, dtype=np.float32)

print(f"총 파라미터 개수: {len(weights)}")
print(f"최솟값: {np.min(weights):.6f}")
print(f"최댓값: {np.max(weights):.6f}")
print(f"평균: {np.mean(weights):.6f}")
print(f"표준편차: {np.std(weights):.6f}")


총 파라미터 개수: 47776
최솟값: -0.825908
최댓값: 2.894141
평균: -0.005734
표준편차: 0.080991
